In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from chart_studio.plotly import plot, iplot
import cufflinks as cf
cf.go_offline(connected=True)

In [2]:
themes = cf.getThemes() #테마목록

cf.set_config_file(theme = 'pearl') #theme설정

# layout = { 'font': { 'family': 'consolas', 'size': 15, 'color': 'black'  #layout설정
#                    } 
#          }
# layout['title'] = "Title" 
# #iplot 안에 layout = layout으로 

In [3]:
price_df = pd.read_csv('./data/kospi/preprocessed/df_return.csv',index_col=0)
df = pd.read_csv('./data/similarity/jaccard.csv',index_col=0)
rm_df = pd.read_csv('./data/kospi/raw/rm_kospi200.csv',index_col=0)

In [4]:
abs_return = price_df.values - rm_df['return'].values[:,np.newaxis]
de_rm_df = pd.DataFrame(data=abs_return, index=price_df.index, columns=price_df.columns)

In [5]:
de_rm_df.head() #kospi200 market return을 감해준 수익률 data

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100104,-0.002808,-0.018529,-0.046101,-0.033509,0.017940,-0.024881,0.030106,0.031618,0.033737,-0.007300,...,-0.002856,0.015088,-0.008779,0.006378,0.001263,0.003794,-0.013965,-0.011555,-7.299270e-07,-0.012325
20100105,-0.002681,0.008590,0.001074,0.000753,-0.006470,-0.041025,-0.052380,-0.030805,-0.028212,-0.030484,...,-0.041339,0.013127,0.007353,0.019400,-0.025747,-0.036277,-0.003533,0.041370,6.571034e-02,-0.008876
20100106,-0.016060,0.003080,0.028553,0.046807,0.011906,-0.014452,0.001966,-0.035338,0.043180,-0.031849,...,-0.031360,-0.016883,0.002112,-0.008212,-0.068328,-0.016369,0.006104,-0.004097,-5.939443e-03,-0.011619
20100107,-0.000886,0.006572,0.018501,0.042569,0.012635,-0.010329,-0.030385,0.022925,0.008845,0.011231,...,0.019695,-0.005453,0.025174,0.001681,-0.007132,0.006732,0.022945,0.010857,3.617400e-03,0.002989
20100108,-0.008403,-0.014540,0.011568,0.058574,0.004365,0.002605,0.015008,-0.014007,0.004146,-0.009838,...,-0.029685,0.060864,-0.010435,-0.003111,-0.021566,0.003850,-0.001080,0.002130,-6.100041e-03,-0.004370


2010년~2011 사이에는 비교할 동일기업 사업보고서 존재하지 않으므로 유사도가 존재하지않는다. 2011년부터 발행되며 filing월은 각자 다르다.

In [10]:
arr = df.values
col_ls = df.columns.values
finite_arr = ~np.isnan(arr)

In [11]:
def converter(idx) :     
    tmp_key = col_ls[finite_arr[idx]]
    tmp_val = arr[idx][finite_arr[idx]]
    tmp_dict = dict(zip(tmp_key,tmp_val))
    return tmp_dict

In [15]:
%%time
container = []
dict_ = dict(zip(col_ls, [np.nan for _ in range(len(col_ls))]))
idx_ls = []
for idx in range(1, len(arr)) : 
    tmp_dict = converter(idx)
    if len(tmp_dict) :
        idx_ls.append(idx)
    container.append(dict_.copy())
    dict_.update(tmp_dict)
container.append(dict_)

assert len(container) == len(df)

Wall time: 20.9 ms


In [16]:
df_converter = lambda x : pd.DataFrame.from_dict(x,orient='index').T
concat_df = pd.concat(map(df_converter, container),ignore_index=True)
concat_df.index = df.index 

In [17]:
ls_ = [] #np.sign이용해 모든 기업에 대하여 총 바뀐 값이 9개(10년치) 인지 확인
for i in concat_df.columns:
    if len(concat_df[np.sign(concat_df).sum(1)==113].loc[:,i].unique()) == 9:
        ls_.append(i)

len(ls_)

C:\Users\yphy0\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning:

invalid value encountered in sign



113

In [18]:
concat_df

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.6787,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.77672,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839
20191228,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.6787,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.77672,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839
20191229,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.6787,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.77672,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839
20191230,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.6787,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.77672,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839


'20110629'가 처음으로 113개 기업의 작년 기준 sim values가 생기는 달. 월별 수익률을 가져올 것므로 2011.07부터 포트폴리오를 구성하겠다

In [19]:
df_ = concat_df.loc[20110701:,:].copy()
price_df = price_df.loc[20110701:,:]
df_.head()

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110701,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.75,0.5762,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.71663
20110702,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.75,0.5762,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.71663
20110703,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.75,0.5762,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.71663
20110704,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.75,0.5762,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.71663
20110705,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.75,0.5762,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.71663


In [20]:
def get_q(df_, lower_bound, upper_bound) : #quntile 설정해주면 해당 quantile에 해당하는 boolean값 반환. cross-sectional 기준
    low_q1_threshold = np.percentile(df_,lower_bound,1)
    low_q1_thres_mat = np.repeat(low_q1_threshold[:,np.newaxis],df_.shape[1],1)

    high_q1_threshold = np.percentile(df_,upper_bound,1)
    high_q1_thres_mat = np.repeat(high_q1_threshold[:,np.newaxis],df_.shape[1],1)
    
    low_bool_mat = low_q1_thres_mat > df_
    high_bool_mat1 = high_q1_thres_mat > df_
    
    low_bool_mat = low_q1_thres_mat > df_
    high_bool_mat = high_q1_thres_mat > df_
    
    return np.bitwise_and(high_bool_mat,~low_bool_mat)

In [21]:
yyyymmdd_ls = list(map(lambda x : int(str(x)[:-2]), get_q(df_,20,40).index))
idx_ls = []
for i in range(len(yyyymmdd_ls)-1) : 
    if yyyymmdd_ls[i] != yyyymmdd_ls[i+1] : 
        idx_ls.append(i)
idx_ls.append(-1) 

monthly_df = df_.loc[df_.index[idx_ls]] #월말 기준으로 앞의 concat_df에서 index만 수정해서 similarity sorting
monthly_df

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110731,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.750000,0.576200,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.716630
20110831,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.750000,0.576200,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.716630
20110930,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.750000,0.576200,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.716630
20111031,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.750000,0.576200,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.716630
20111130,0.482558,0.852459,0.615385,0.579749,0.630503,0.203113,0.705706,0.534641,0.750000,0.576200,...,0.873362,0.663158,0.903475,0.462889,0.577154,0.713235,0.633452,0.574394,0.755193,0.716630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190831,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.678700,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.776720,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839
20190930,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.678700,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.776720,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839
20191031,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.678700,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.776720,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839
20191130,0.690635,0.532609,0.727488,0.679662,0.510638,0.625402,0.678700,0.738854,0.694118,0.811252,...,0.758364,0.564672,0.671296,0.776720,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839


In [22]:
def get_monthly_return(df): #월별 수익률 생성
    ex_ls = ['0'+str(x) for x in np.arange(7,10)]+['10','11','12'] #201107~201112의 월별 수익률
    month_ls = ['0'+str(x) for x in np.arange(1,10)]+['10','11','12'] #201201~201912 까지의 월별 수익률
    year_ls = [str(x) for x in np.arange(2,10)]

    ai = []
    for i in ex_ls:
        ai.append((1+df.loc['2011'+i+'01':'2011'+i+'31']).prod()-1)

    ae = []
    for i in year_ls:
        for j in month_ls:
            ae.append((1+df.loc['201'+i+j+'01':'201'+i+j+'31']).prod() -1)

    month_return_ls = ai+ae
    df_ = pd.DataFrame(month_return_ls,index=monthly_df.index)
    return df_

In [54]:
month_return_df = get_monthly_return(price_df) 
month_return_df

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110731,0.109527,8.518192e-02,-6.481481e-02,0.183998,-0.019455,-0.058166,0.070539,0.190727,-0.031936,0.136449,...,0.060519,0.155125,0.160051,0.108579,0.010098,0.121573,-0.066255,0.087282,0.299363,0.039604
20110831,-0.051503,-1.672363e-01,-1.277228e-01,-0.087834,-0.083333,-0.071126,-0.087855,-0.125665,-0.212371,-0.042763,...,-0.021739,-0.071942,-0.203091,-0.130593,-0.031667,0.038257,-0.105926,-0.224771,0.068627,-0.160000
20110930,-0.268778,-1.147603e-01,-2.429058e-01,0.007399,-0.164502,0.016318,0.018414,-0.088048,0.117801,-0.113402,...,-0.066667,-0.108527,-0.203601,-0.026426,0.012045,-0.171955,0.011094,-0.159763,-0.100917,-0.085034
20111031,0.001240,1.342738e-01,1.499250e-01,0.047802,0.015544,0.106208,0.005563,0.013309,0.070258,0.115310,...,-0.041667,0.085507,0.135652,0.152857,0.068027,0.114957,0.119848,0.200704,0.047619,0.075589
20111130,0.000000,2.220446e-16,-7.301173e-02,-0.017546,-0.021429,0.016723,-0.034578,0.030671,0.004376,-0.044309,...,0.006211,-0.010681,-0.111792,-0.021066,-0.113052,0.034368,-0.009798,-0.049853,-0.071429,-0.130184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190831,-0.136298,6.004564e-02,4.379562e-02,-0.042860,-0.073934,-0.048387,-0.001143,0.005714,0.006502,-0.168390,...,0.052632,-0.029530,-0.045726,-0.064748,-0.071429,-0.212245,0.024775,-0.035088,-0.198246,-0.085253
20190930,-0.003322,-4.355861e-03,1.048951e-02,0.054732,0.064483,-0.030508,0.043478,-0.006818,0.062016,0.079929,...,0.001282,0.041494,0.047917,0.084615,0.080769,0.046632,-0.068132,0.006061,0.026258,0.075567
20191031,0.056667,-1.750424e-02,8.996540e-02,-0.068263,-0.125000,-0.069930,-0.066886,0.058352,-0.002433,0.001645,...,-0.053777,-0.099602,-0.012922,-0.043972,0.366548,-0.027228,-0.004717,-0.036145,-0.025586,-0.017564
20191130,0.037855,-3.786560e-02,2.220446e-16,-0.123393,-0.012088,0.078947,0.016451,0.108108,-0.013415,0.036125,...,0.125846,0.057522,-0.036254,0.060831,-0.015625,-0.089059,-0.072275,-0.084375,0.085339,0.097735


In [24]:
get_q(monthly_df,80,100) #Q5 boolean 

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110731,False,True,False,False,False,False,False,False,True,False,...,True,False,True,False,False,False,False,False,True,False
20110831,False,True,False,False,False,False,False,False,True,False,...,True,False,True,False,False,False,False,False,True,False
20110930,False,True,False,False,False,False,False,False,True,False,...,True,False,True,False,False,False,False,False,True,False
20111031,False,True,False,False,False,False,False,False,True,False,...,True,False,True,False,False,False,False,False,True,False
20111130,False,True,False,False,False,False,False,False,True,False,...,True,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190831,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,True,False,True
20190930,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,True,False,True
20191031,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,True,False,True
20191130,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,True,True,False,True


In [25]:
np.nanmean(month_return_df[get_q(monthly_df,0,20)])

0.0021510993424200767

In [27]:
values = []
for i in range(0,100,20) : 
    values.append(np.nanmean(month_return_df[get_q(monthly_df,i,i+20)]))

calendar_df = pd.DataFrame(values,index=['Q1','Q2','Q3','Q4','Q5'],columns=['returns'])
calendar_df

,returns
Q1,0.002151
Q2,0.002903
Q3,0.003663
Q4,-0.000203
Q5,0.001781


In [55]:
calendar_df.iplot(kind='line')

In [56]:
de_rm_df

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100104,-0.002808,-0.018529,-0.046101,-0.033509,0.017940,-0.024881,0.030106,0.031618,0.033737,-0.007300,...,-0.002856,0.015088,-0.008779,0.006378,0.001263,0.003794,-0.013965,-0.011555,-7.299270e-07,-0.012325
20100105,-0.002681,0.008590,0.001074,0.000753,-0.006470,-0.041025,-0.052380,-0.030805,-0.028212,-0.030484,...,-0.041339,0.013127,0.007353,0.019400,-0.025747,-0.036277,-0.003533,0.041370,6.571034e-02,-0.008876
20100106,-0.016060,0.003080,0.028553,0.046807,0.011906,-0.014452,0.001966,-0.035338,0.043180,-0.031849,...,-0.031360,-0.016883,0.002112,-0.008212,-0.068328,-0.016369,0.006104,-0.004097,-5.939443e-03,-0.011619
20100107,-0.000886,0.006572,0.018501,0.042569,0.012635,-0.010329,-0.030385,0.022925,0.008845,0.011231,...,0.019695,-0.005453,0.025174,0.001681,-0.007132,0.006732,0.022945,0.010857,3.617400e-03,0.002989
20100108,-0.008403,-0.014540,0.011568,0.058574,0.004365,0.002605,0.015008,-0.014007,0.004146,-0.009838,...,-0.029685,0.060864,-0.010435,-0.003111,-0.021566,0.003850,-0.001080,0.002130,-6.100041e-03,-0.004370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191223,0.010850,0.009773,-0.011539,0.008582,0.001490,-0.008927,0.002609,0.015574,-0.002721,-0.001695,...,0.000278,0.008377,0.007193,0.005381,-0.023324,-0.001257,-0.025570,0.001490,9.771533e-03,0.018819
20191224,0.022137,0.000521,-0.003220,0.006681,0.000131,0.006681,0.005564,0.011311,-0.001776,0.027448,...,-0.001814,0.005313,0.012352,-0.004947,0.014314,0.020455,0.020587,0.009938,5.207457e-04,-0.002337
20191226,0.008216,0.012567,-0.003960,0.010124,-0.006158,0.010075,0.004988,0.000648,0.006701,-0.016480,...,-0.002736,-0.001221,-0.003960,0.006497,-0.011536,0.009627,-0.010195,0.002533,1.718354e-04,-0.004972
20191227,-0.006884,0.000370,0.026123,-0.027488,-0.025903,-0.024638,-0.020506,-0.026813,0.008781,-0.008631,...,-0.005099,-0.014806,-0.022674,-0.033631,-0.011510,-0.028005,0.013689,-0.029683,2.492982e-02,-0.031205


## Figure 7 Event time Returns

This figure plots the weighted average cumulative abnormal return for the top (highest similarity) and
bottom (lowest similarity) quintile portfolios.

naive하게 3월말에 포트폴리오를 확정해서 추이를 살펴본다고 해보자. 실제로 월별 변경내용을 확인하면 대부분이 3월, 간혹 4월 존재

2011년 6월에 최종수치변경되므로 2012년 3월말부터 해본다

In [30]:
monthly_df.loc[20150131:,:].diff()
monthly_df.head().diff()

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20110831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20110930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20111031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20111130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
de_rm_df.head()

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100104,-0.002808,-0.018529,-0.046101,-0.033509,0.017940,-0.024881,0.030106,0.031618,0.033737,-0.007300,...,-0.002856,0.015088,-0.008779,0.006378,0.001263,0.003794,-0.013965,-0.011555,-7.299270e-07,-0.012325
20100105,-0.002681,0.008590,0.001074,0.000753,-0.006470,-0.041025,-0.052380,-0.030805,-0.028212,-0.030484,...,-0.041339,0.013127,0.007353,0.019400,-0.025747,-0.036277,-0.003533,0.041370,6.571034e-02,-0.008876
20100106,-0.016060,0.003080,0.028553,0.046807,0.011906,-0.014452,0.001966,-0.035338,0.043180,-0.031849,...,-0.031360,-0.016883,0.002112,-0.008212,-0.068328,-0.016369,0.006104,-0.004097,-5.939443e-03,-0.011619
20100107,-0.000886,0.006572,0.018501,0.042569,0.012635,-0.010329,-0.030385,0.022925,0.008845,0.011231,...,0.019695,-0.005453,0.025174,0.001681,-0.007132,0.006732,0.022945,0.010857,3.617400e-03,0.002989
20100108,-0.008403,-0.014540,0.011568,0.058574,0.004365,0.002605,0.015008,-0.014007,0.004146,-0.009838,...,-0.029685,0.060864,-0.010435,-0.003111,-0.021566,0.003850,-0.001080,0.002130,-6.100041e-03,-0.004370


In [33]:
values = []
for i in range(0,100,20) : 
    values.append(np.nanmean(month_abnormal_df[get_q(monthly_df,i,i+20)]))

In [38]:
def get_event_returns(df,holding_period):
    iloc_ls = [i for i in range(8,102,12)] 
    values_ls = []
    for i in iloc_ls:
        values_ls.append((1+month_abnormal_df.iloc[i:i+holding_period,:]).prod()-1) #8년치 수익률저장

    values = []
    for idx, i in enumerate(iloc_ls):
        for j in range(0,100,20): #Q1~Q5
            display(values_ls[idx])
            values.append(values_ls[idx][get_q(monthly_df,j,j+20).iloc[i,:]].mean()) #(2012~2019) x (Q1~Q5) 의 40개

    result = [values[i * 5:(i + 1) * 5] for i in range((len(values) + 5 - 1) // 5 )] #8년치 Q1~Q5를 5쿼터씩 리스트로 생성

    q_1 = np.mean([result[i][0] for i in range(0,len(result))])
    q_2 = np.mean([result[i][1] for i in range(0,len(result))])
    q_3 = np.mean([result[i][2] for i in range(0,len(result))])
    q_4 = np.mean([result[i][3] for i in range(0,len(result))])
    q_5 =np.mean([result[i][4] for i in range(0,len(result))])

    return(q_1,q_5)
    

In [40]:
ls = [get_event_returns(month_abnormal_df,i) for i in range(1,11+1)]

삼양홀딩스      0.011856
유한양행      -0.011105
CJ대한통운    -0.022885
두산        -0.118193
대림산업      -0.024651
             ...   
LF        -0.048361
후성        -0.011646
SK이노베이션   -0.106076
CJ제일제당     0.046697
KB금융       0.005831
Length: 113, dtype: float64

삼양홀딩스      0.011856
유한양행      -0.011105
CJ대한통운    -0.022885
두산        -0.118193
대림산업      -0.024651
             ...   
LF        -0.048361
후성        -0.011646
SK이노베이션   -0.106076
CJ제일제당     0.046697
KB금융       0.005831
Length: 113, dtype: float64

삼양홀딩스      0.011856
유한양행      -0.011105
CJ대한통운    -0.022885
두산        -0.118193
대림산업      -0.024651
             ...   
LF        -0.048361
후성        -0.011646
SK이노베이션   -0.106076
CJ제일제당     0.046697
KB금융       0.005831
Length: 113, dtype: float64

삼양홀딩스      0.011856
유한양행      -0.011105
CJ대한통운    -0.022885
두산        -0.118193
대림산업      -0.024651
             ...   
LF        -0.048361
후성        -0.011646
SK이노베이션   -0.106076
CJ제일제당     0.046697
KB금융       0.005831
Length: 113, dtype: float64

삼양홀딩스      0.011856
유한양행      -0.011105
CJ대한통운    -0.022885
두산        -0.118193
대림산업      -0.024651
             ...   
LF        -0.048361
후성        -0.011646
SK이노베이션   -0.106076
CJ제일제당     0.046697
KB금융       0.005831
Length: 113, dtype: float64

...

삼양홀딩스     -0.233463
유한양행      -0.087414
CJ대한통운    -0.216561
두산        -0.117309
대림산업      -0.065318
             ...   
LF        -0.299988
후성        -0.035100
SK이노베이션   -0.230418
CJ제일제당    -0.245889
KB금융       0.035936
Length: 113, dtype: float64

삼양홀딩스     -0.233463
유한양행      -0.087414
CJ대한통운    -0.216561
두산        -0.117309
대림산업      -0.065318
             ...   
LF        -0.299988
후성        -0.035100
SK이노베이션   -0.230418
CJ제일제당    -0.245889
KB금융       0.035936
Length: 113, dtype: float64

삼양홀딩스     -0.233463
유한양행      -0.087414
CJ대한통운    -0.216561
두산        -0.117309
대림산업      -0.065318
             ...   
LF        -0.299988
후성        -0.035100
SK이노베이션   -0.230418
CJ제일제당    -0.245889
KB금융       0.035936
Length: 113, dtype: float64

삼양홀딩스     -0.233463
유한양행      -0.087414
CJ대한통운    -0.216561
두산        -0.117309
대림산업      -0.065318
             ...   
LF        -0.299988
후성        -0.035100
SK이노베이션   -0.230418
CJ제일제당    -0.245889
KB금융       0.035936
Length: 113, dtype: float64

삼양홀딩스     -0.233463
유한양행      -0.087414
CJ대한통운    -0.216561
두산        -0.117309
대림산업      -0.065318
             ...   
LF        -0.299988
후성        -0.035100
SK이노베이션   -0.230418
CJ제일제당    -0.245889
KB금융       0.035936
Length: 113, dtype: float64

In [41]:
ls

[(0.0019825508113308896, -0.005666672110517132),
 (0.011018855572727242, 0.009810245711390744),
 (0.01380182364358497, 0.0010867654164891877),
 (0.008343881012831613, 0.0013520473261310444),
 (0.005053705494621558, 0.009143353278284236),
 (0.0047506217324005204, 0.0001572303320464749),
 (0.012338720574706275, 0.0021728111431510294),
 (-0.01010902073007411, -0.016547857862935587),
 (-0.0008995229571131351, -0.012371983838754382),
 (-0.0043796855047439215, -0.0014164545086230718),
 (0.010963304691267023, -0.009766265914959563)]

In [42]:
q1 = [i[0] for i in ls]
q5= [i[1] for i in ls]
dd = pd.DataFrame([q1,q5]).T
dd.columns = ['Q1','Q5']
dd.iplot()

In [43]:
result = [values[i * 5:(i + 1) * 5] for i in range((len(values) + 5 - 1) // 5 )] #8년치 Q1~Q5를 5쿼터씩 리스트로 생성

q_1 = np.mean([result[i][0] for i in range(0,len(result))])
q_2 = np.mean([result[i][1] for i in range(0,len(result))])
q_3 = np.mean([result[i][2] for i in range(0,len(result))])
q_4 = np.mean([result[i][3] for i in range(0,len(result))])
q_5 =np.mean([result[i][4] for i in range(0,len(result))])

In [ ]:
de

In [113]:
g=[]
for idx,i in enumerate(de_rm_df.index):
    if str(i)[5:] == '330':
        print(i)

20100330
20110330
20120330
20150330
20160330
20170330
20180330


In [302]:
ret_df = de_rm_df.loc[20120300:].copy()
sim_df = concat_df.loc[ret_df.index].copy()

ret_df.shape, sim_df.shape

((1926, 113), (1926, 113))

In [303]:
ret_df

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20120302,0.017876,0.007172,0.026851,-0.001198,0.002770,-0.002376,0.012946,-0.003554,0.007080,0.048763,...,0.001796,0.014272,0.011186,-0.012605,-0.008993,0.015469,0.013098,0.001497,0.037682,0.004870
20120305,-0.009958,-0.014797,0.006537,0.001348,-0.037335,0.003112,0.008701,-0.001699,-0.003041,-0.021131,...,0.030991,0.018800,-0.003666,0.012659,0.036621,-0.019179,0.004486,-0.019475,0.028059,-0.001967
20120306,0.012634,0.007177,-0.019014,-0.004583,0.007177,0.036588,0.012763,0.021502,-0.014454,-0.012010,...,0.001329,-0.020871,-0.006777,0.025079,0.018657,-0.018155,-0.014085,-0.009444,-0.026647,-0.005033
20120307,0.000013,-0.007514,0.002176,-0.002395,0.005362,-0.021368,-0.004378,-0.012836,-0.000693,-0.031177,...,0.012452,-0.024895,0.001649,-0.006821,0.013294,0.052828,0.010989,-0.013024,0.000379,0.007039
20120308,-0.010972,-0.031247,-0.007139,-0.003577,0.007064,-0.014318,0.011524,-0.005995,0.012734,-0.071593,...,-0.000805,0.024880,0.007830,0.028712,-0.013372,-0.029769,-0.022735,0.004807,-0.017283,0.011463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191223,0.010850,0.009773,-0.011539,0.008582,0.001490,-0.008927,0.002609,0.015574,-0.002721,-0.001695,...,0.000278,0.008377,0.007193,0.005381,-0.023324,-0.001257,-0.025570,0.001490,0.009772,0.018819
20191224,0.022137,0.000521,-0.003220,0.006681,0.000131,0.006681,0.005564,0.011311,-0.001776,0.027448,...,-0.001814,0.005313,0.012352,-0.004947,0.014314,0.020455,0.020587,0.009938,0.000521,-0.002337
20191226,0.008216,0.012567,-0.003960,0.010124,-0.006158,0.010075,0.004988,0.000648,0.006701,-0.016480,...,-0.002736,-0.001221,-0.003960,0.006497,-0.011536,0.009627,-0.010195,0.002533,0.000172,-0.004972
20191227,-0.006884,0.000370,0.026123,-0.027488,-0.025903,-0.024638,-0.020506,-0.026813,0.008781,-0.008631,...,-0.005099,-0.014806,-0.022674,-0.033631,-0.011510,-0.028005,0.013689,-0.029683,0.024930,-0.031205


In [304]:
yyyymmdd_ls = [str(i)[:-2] for i in sim_df.index]
idx_ls = []
for idx in range(0,len(yyyymmdd_ls)-1) : 
    if yyyymmdd_ls[idx] != yyyymmdd_ls[idx+1] : 
        idx_ls.append(idx)
idx_ls.append(-1)

monthly_sim_df = sim_df.iloc[idx_ls]
monthly_sim_df.shape

(94, 113)

In [518]:
def get_q_returns(days):
    march_idx =[ x for x in range(0,94,12)] #3월에 해당되는 idx 추출
    monthly_sim_df.iloc[march_idx,:]
    q1_march_values = [get_q(monthly_sim_df,0,20).iloc[x,:].values for x in march_idx] #3월말 값들의 Q1 여부 8년치
    q2_march_values = [get_q(monthly_sim_df,20,40).iloc[x,:].values for x in march_idx] #3월말 값들의 Q2 여부 8년치
    q3_march_values = [get_q(monthly_sim_df,40,60).iloc[x,:].values for x in march_idx] #3월말 값들의 Q3 여부 8년치
    q4_march_values = [get_q(monthly_sim_df,60,80).iloc[x,:].values for x in march_idx] #3월말 값들의 Q4 여부 8년치
    q5_march_values = [get_q(monthly_sim_df,80,100).iloc[x,:].values for x in march_idx] #3월말 값들의 Q5 여부 8년치
    
    re_march_idx = [idx_ls[i] for i in range(0,len(idx_ls),12)] #return_df 의 3월말 idx값들 8년치

    q1 = np.mean([((1+ret_df.iloc[i:i+days,:].cumsum())-1).iloc[-1,:][q1_march_values[idx]].mean() for idx,i in enumerate(re_march_idx)])
    q2 = np.mean([((1+ret_df.iloc[i:i+days,:].cumsum())-1).iloc[-1,:][q2_march_values[idx]].mean() for idx,i in enumerate(re_march_idx)])
    q3 = np.mean([((1+ret_df.iloc[i:i+days,:].cumsum())-1).iloc[-1,:][q3_march_values[idx]].mean() for idx,i in enumerate(re_march_idx)])
    q4 = np.mean([((1+ret_df.iloc[i:i+days,:].cumsum())-1).iloc[-1,:][q4_march_values[idx]].mean() for idx,i in enumerate(re_march_idx)])
    q5 = np.mean([((1+ret_df.iloc[i:i+days,:].cumsum())-1).iloc[-1,:][q5_march_values[idx]].mean() for idx,i in enumerate(re_march_idx)])
        
    df_ = pd.DataFrame([q1,q5],index=['Q1','Q5'])
    
    return df_

In [534]:
six_month_df

,Q1,Q5
1m,0.011513,0.015792
2m,0.014704,0.016976
3m,0.011720,0.014317
4m,0.014312,0.020536
5m,0.009526,0.015260
6m,0.012450,0.012493


In [565]:
ee = pd.DataFrame([0,0],index=['Q1','Q5'])
six_month_df = pd.concat([get_q_returns(i) for i in range(20,140,20)],axis=1)
six_month_df = pd.concat([ee,six_month_df],axis=1).T
six_month_df.index = ['0','1','2','3','4','5','6']

In [568]:
six_month_df.iplot(kind='line',title = 'CAR for holding periods' ,xTitle='Month', yTitle = 'CAR')